In [1]:
import os
import sys

from pathlib import Path

In [2]:
sys.path.append("../pymol_docking_plugin")

In [3]:
from client import PyMOLDockingClient

In [4]:
client = PyMOLDockingClient()

In [ ]:
client.check_health()

In [6]:
prot ="./LAC3.pdb"
lig = "./Lig_Min.sdf"
# lig = "c1ccccc1"
crystal = "./Crystal.sdf"

In [7]:
# prot = "./8gcy.pdb"
# lig = "./8gcy_Crystal.sdf"
# # lig = "c1ccccc1"
# crystal = "./8gcy_Crystal.sdf"

In [8]:
# client.dock_minimize(
#     protein_file=prot,
#     ligand=lig,
#     crystal_file=crystal,
#     dock_mode="Minimize",
#     output_name="TMP",
#     minimize=False,
#     is_smiles=False
# )

In [ ]:
client.inplace_minimization(
    protein_file=prot,
    ligand=lig,
    output_dir="."
)

---

In [2]:
import numpy as np
import io
from tempfile import NamedTemporaryFile
from pdbfixer import PDBFixer
from typing import List, Dict, Any, Optional, Union, Tuple, TypeVar, IO, BinaryIO, TextIO
from pathlib import Path
import os
import tempfile

# Biotite
import biotite.structure.io.pdb as pdb
from biotite.structure import AtomArray

# RDKit
from rdkit import Chem
from rdkit.Chem import AllChem

# OpenMM Application Layer
from openmm import app
from openmm.app import (
    Modeller, Simulation, PDBFile, DCDReporter, 
    StateDataReporter, CheckpointReporter
)

# OpenMM Library Layer
from openmm import (
    Platform, LangevinIntegrator, MonteCarloBarostat,
    CustomExternalForce, State, System, Context
)

# OpenMM Units
from openmm import unit
from openmm.unit import Quantity
import datamol as dm

# OPENFF
from openff.toolkit import Molecule
from openmmforcefields.generators import SystemGenerator

# Type for OpenMM unit quantities
UnitQuantity = Quantity

In [7]:
from openmmforcefields.generators import (
    EspalomaTemplateGenerator,
)

In [8]:
prot ="./LAC3.pdb"
lig = "./Lig_Min.sdf"

lig_mol = dm.read_sdf(lig)[0]

In [9]:
fixer: PDBFixer = PDBFixer(str(prot))
fixer.removeHeterogens(keepWater=False)
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()

fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.4)

# Parse the ligand
ligand_mol: Molecule = Molecule.from_rdkit(lig_mol)
lig_top = ligand_mol.to_topology()

# Merge the ligand into the protein
modeller: Modeller = Modeller(fixer.topology, fixer.positions)
modeller.add(lig_top.to_openmm(), lig_top.get_positions().to_openmm())

In [5]:
# Create the forcefield
forcefield_kwargs: Dict[str, Any] = { 
    'constraints': app.HBonds, 
    'hydrogenMass': 4*unit.amu 
}

In [ ]:

# Set up the system generator with appropriate forcefields
system_generator: SystemGenerator = SystemGenerator(
    forcefields=['amber/ff14SB.xml', 'amber/tip3p_standard.xml'],
    small_molecule_forcefield='espaloma-0.3.2',
    molecules=[ligand_mol],
    forcefield_kwargs=forcefield_kwargs
)